# Simulation for Potts model
- All levels of interaction
- Field (constant for each point)
- Preference for low number of colors
- Distance between different colors (0 if equal, 1 of different)

- $N$ vertices
- $P(\sigma) = \frac{1}{N^N} \exp((N-n(\sigma))\gamma$), where $n(\sigma) = \text{# of colors used in } \sigma$

- Consider the interaction on the largest hyperedge on lattice $\Lambda$, where $|\Lambda| = N$
- We assign $\eta_{\Lambda} \in \mathcal{P}(\{1,2,...,q\}^N)$ which is in one-to-one correspondence with $\{1,2,...,N\}$ (a set with $k$ colors corresponds to $k$).
- Each of these corresponds to an energy level in $\{(N-1)\gamma, (N-2)\gamma, ..., 0\}$


- Consider the interaction between adjacent sites (edges) $b$ with size $|b|=2$
- To each edge assign $\eta_{b} \in \mathcal{P}(\{0,1\}^2)$ which is in one-to-one correspondence with the edge being open or closed.
- Each of these corresponds to an energy level in $\{J, -J\}$

- Consider the interaction between adjacent sites and the field, corresponding to $v$ with size $|v|=1$
- To each site assign $\eta_{v} \in \mathcal{P}(\{1,2,...,q\}^N)$ which is in one-to-one correspondence with the color of the site.
- Each of these corresponds to an energy level in $\{(N-1)\alpha, (N-2)\alpha, ...,(N-k)\alpha, ...,0\}$ where $k$ corresponds to the $k$th preferred color of the field.

- Take $ \upsilon_k =  \frac{e^{((N-k)\gamma)} - e^{((N-k-1)\gamma)}}{e^{((N-1)\gamma)}} $

- Joint distribution
- $Q(\eta, \sigma) = \upsilon_{\eta_{\Lambda}} \frac{1}{N^N} \mathbb{1}_{(n(\sigma) \leq \eta_{\Lambda})} $

- Marginals
- $Q(\eta | \sigma) =  \frac { \upsilon_{\eta_{\Lambda}} \frac{1}{N^N} \mathbb{1}_{(n(\sigma) \leq \eta_{\Lambda})} }{P(\sigma)}$
- $Q(\sigma | \eta) = \frac{1}{|\{\sigma:  \eta(\sigma) \leq \eta(\Lambda)\}|} $

In [1]:
# Initial code from https://rajeshrinet.github.io/blog/2014/ising-model/

import numpy as np 
from numpy.random import rand
from scipy.special import binom

def initial_config(N, no_colors=2):   
    ''' Generate a random color/spin configuration for initialization'''
    state = np.random.randint(no_colors, size=(N,N))
    return state


def mc_move(config, eta_prob, parts, parts_prob, N, no_colors):
    '''Monte Carlo move using generalized SW algorithm '''
    # Assign eta to each hyperedge
    eta = assign_etas(config, eta_prob, no_colors)
#     print('assigned eta:', eta)

    # Assign labels to each site
    config = assign_labels(eta, parts, parts_prob, N, no_colors)
#     print('assigned configuration:\n', config)

    return config


def number_of_colors (config):
    '''Number of colors used in the given confuration of labels'''
    return len(np.unique(config))


def avg_sites_per_color (config):
    '''Average number of sites each present color has in the configuration'''
    unq = np.unique(config)
    return config.shape[0]*config.shape[1]/len(unq)


def prob_eta_lambda(gamma, no_colors, k):
    '''Probability of the eta lambda corresponding to a certain number of colors'''
    if k < no_colors:
        return ( np.exp((no_colors-k)*gamma) - np.exp((no_colors-k-1)*gamma) ) / np.exp((no_colors-1)*gamma)
    elif k == no_colors:
        return 1 / np.exp((no_colors-1)*gamma)
    
    
def prob_eta_edge(J, col1, col2):
    '''Probability of the eta edge corresponding to open or not'''
    if col1==col2:
        return 1 - np.exp(-2J)
    else:
        return np.exp(-2J)
    
    
def prob_eta_site(alpha, no_colors, k):
    '''Probability of the eta site corresponding to a maximum color number (lower have higher energy)'''
    if k < no_colors:
        return ( np.exp((no_colors-k)*alpha) - np.exp((no_colors-k-1)*alpha) ) / np.exp((no_colors-1)*alpha)
    elif k == q:
        return 1 / np.exp((no_colors-1)*alpha)

    
def lengths_of_partition(partition):
    '''Lengths of equal subtrings in a partition'''
    lengths = []
    cnt = 0
    curr = partition[0]
    for i in range(len(partition)):
        if partition[i] == curr:
            cnt += 1
            if i == len(partition)-1:
                lengths.append(cnt)
        else:
            lengths.append(cnt)
            cnt = 1
            curr = partition[i]
            if i == len(partition)-1:
                lengths.append(cnt)
    return tuple(lengths)


def multinomial(params):
    if len(params) == 1:
        return 1
#     print (sum(params), params[0])
    return binom(sum(params), params[0]) * multinomial(params[1:])


def prob_of_partition(partition, no_sites, no_colors):
    '''Probability of a given partition'''
    p = partition
    l = ll = lengths_of_partition(p)
    if sum(l) < no_colors: ll = l + (no_colors-sum(l),)
    m1 = multinomial(ll)
    m2 = multinomial(p)
#     print(p, m1, m2, m1*m2)
#     print(l)
#     print(ll)
#     print()
    return m1*m2

    
def assign_etas(config, prob, no_colors):
    '''Assign a number of colors that is at least the current number of colors, with correct probabilities'''
    current_k = number_of_colors(config)
#     print('current no. of colors:', current_k)
    p = prob[current_k-1:]
#     print('probabilities for no. of colors:', p)
    p = p / p.sum()
# #     print('normalized:', p)
    eta_lambda = np.random.choice((np.arange(current_k, no_colors+1)), p=p)

    '''Assign a closed or open bond to each edge'''
    # loop over edges
    current_k = number_of_colors(config)
#     print('current no. of colors:', current_k)
    p = prob[current_k-1:]
#     print('probabilities for no. of colors:', p)
    p = p / p.sum()
# #     print('normalized:', p)
    eta_lambda = np.random.choice((np.arange(current_k, no_colors+1)), p=p)
    
    return eta_lambda, eta_edge, eta_site
    
def assign_labels(max_colors, parts, prob, N, q):
    '''Assign a color configuration that has at most the corresponding number of eta'''
    # Brute force version
#     config = initial_config(N, q)
#     while number_of_colors(config) > max_colors:
#         config = initial_config(N, q)
#     return config
    
    idx_max_color = len(parts)                  # index of first occurence of a partition with more than max_colors
    for i in range(len(parts)):
        if len(parts[i]) > max_colors:
            idx_max_color = i
            break
    
#     print('max colors:', max_colors)
#     print('partitions:', parts)
#     print('idx of partition with max color:', idx_max_color)
    p = np.array(prob[:idx_max_color])
#     print('probabilities for partitions:', p)
    p = p / p.sum()
#     print('normalized:', p)
    if idx_max_color > 1: chosen_part = np.random.choice(parts[:idx_max_color], p=p)
    else: chosen_part = parts[0]
#     print('chosen partition:', chosen_part)
        
    # Choose the colors to be used
    chosen_colors = np.random.choice(q, len(chosen_part), replace=False)
#     print('chosen colors:', chosen_colors)

    # Choose a random permutation of the given word
    color_arr = []
    for i in range(len(chosen_part)): color_arr += [chosen_colors[i]]*chosen_part[i]
    color_arr = np.array(color_arr)
#     print('color array:\n', color_arr)
    config = np.random.permutation(color_arr).reshape(N,N)
    return config

In [2]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr

def init_partitions(N, q):
    session = WolframLanguageSession()
    parts = session.evaluate(wl.IntegerPartitions(N*N,q))
#     print(parts)

    sorted_p = tuple(sorted(parts, key=lambda x: len(x)))
#     print(sorted_p)

    part_prob = tuple([prob_of_partition(p, N*N, q) for p in sorted_p])
#     print(part_prob)
#     print(sum(part_prob), q**(N*N))
#     print(q**(N*N)-sum(part_prob))
    part_prob = tuple(part_prob / sum(part_prob))
#     print(part_prob)
    return sorted_p, part_prob

    session.terminate()

In [3]:
# Small experiment

N, q = 3, 4
sorted_p, part_prob = init_partitions(N, q)
gamma = 2.5
print('Initialization done')
print()

eta_prob = np.zeros(q)
for j in range(q):
    eta_prob[j] = prob_eta(gamma,q,j+1)
print('eta probabilities:', eta_prob)

config = initial_config(N,q)
# config = np.array([[0,0],[0,0]])
print('initial config:\n', config)
config = mc_move(config, eta_prob, sorted_p, part_prob, N, q)

Initialization done

eta probabilities: [9.17915001e-01 7.53470516e-02 6.18486263e-03 5.53084370e-04]
initial config:
 [[3 2 2]
 [2 0 0]
 [1 1 1]]


In [93]:
np.random.choice(4, 3, p=eta_prob)

array([0, 0, 0], dtype=int64)